In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [5]:
locations_path = "./DataExpo2018/locations.csv"
forecast_path = "./DataExpo2018/forecast.dat"
hist_path = "./DataExpo2018/histWeather.csv"
output_csv = "./DataExpo2018/forecasts_vs_observed.csv"

In [8]:
locations = pd.read_csv(locations_path)
locations.columns = [c.strip() for c in locations.columns]  # clean column names

# Ensure ordering matches city_id in forecast.dat (1-indexed)
locations = locations.reset_index(drop=True)
locations

,city,state,longitude,latitude,AirPtCd
0,Eastport,Maine,-66.9900,44.9100,KBHB
1,Bangor,Maine,-68.7833,44.8000,KBGR
2,Portland,Maine,-70.2500,43.6667,KPWM
3,Boston,Massachusetts,-71.0833,42.3500,KBOS
4,Providence,Rhode Island,-71.4000,41.8333,KPVD
...,...,...,...,...,...
108,Portland,Oregon,-122.6830,45.5167,KVUO
109,Eugene,Oregon,-123.0830,44.0500,KEUG
110,Hoquiam,Washington,-123.9000,46.9833,KHQM
111,Anchorage,Alaska,-149.9000,61.2167,PAMR


In [9]:
rows = []
with open(forecast_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) < 5:
            continue
        city_id = int(parts[0])
        target_date = parts[1]
        value = parts[2]
        var = parts[3]
        issue_date = parts[4]

        # Map city_id to location
        try:
            loc = locations.iloc[city_id - 1]  # forecast.dat is 1-indexed
        except IndexError:
            continue

        city = loc['city']
        state = loc['state']
        airpt = loc.get('AirPtCd', None)

        rows.append({
            "city_id": city_id,
            "city": city,
            "state": state,
            "airptcd": airpt,
            "target_date": target_date,
            "forecast_issue_date": issue_date,
            "forecast_var": var,
            "forecast_value": value
        })

forecast_df = pd.DataFrame(rows)

# Keep only probability of precipitation forecasts
forecast_df = forecast_df[forecast_df['forecast_var'].str.lower().str.contains('probprecip')]
forecast_df['forecast_value'] = pd.to_numeric(forecast_df['forecast_value'], errors='coerce')

In [12]:
forecast_df

,city_id,city,state,airptcd,target_date,forecast_issue_date,forecast_var,forecast_value
9,1,Eastport,Maine,KBHB,2014-07-09,2014-07-09,ProbPrecip,0
10,1,Eastport,Maine,KBHB,2014-07-09,2014-07-09,ProbPrecip,0
11,1,Eastport,Maine,KBHB,2014-07-10,2014-07-09,ProbPrecip,0
12,1,Eastport,Maine,KBHB,2014-07-10,2014-07-09,ProbPrecip,0
13,1,Eastport,Maine,KBHB,2014-07-11,2014-07-09,ProbPrecip,0
...,...,...,...,...,...,...,...,...
3191967,113,Honolulu,Hawaii,PHNL,2017-09-05,2017-09-01,ProbPrecip,20
3191968,113,Honolulu,Hawaii,PHNL,2017-09-05,2017-09-01,ProbPrecip,20
3191969,113,Honolulu,Hawaii,PHNL,2017-09-06,2017-09-01,ProbPrecip,40
3191970,113,Honolulu,Hawaii,PHNL,2017-09-06,2017-09-01,ProbPrecip,50


In [11]:
hist = pd.read_csv(hist_path)
hist.columns = [c.strip() for c in hist.columns]

# Parse date
date_col = None
for c in ['Date','date','DATE']:
    if c in hist.columns:
        date_col = c
        break
if date_col is None:
    raise ValueError("No date column found in histWeather.csv")
hist[date_col] = pd.to_datetime(hist[date_col])
hist = hist.rename(columns={date_col: 'obs_date'})

# Find precipitation column
precip_col = None
for c in hist.columns:
    if c.lower().startswith('precip'):
        precip_col = c
        break
if precip_col is None:
    raise ValueError("No precipitation column found in histWeather.csv")
hist['precip_in'] = pd.to_numeric(hist[precip_col].replace('T','0.0'), errors='coerce')

hist_df = hist[['AirPtCd', 'obs_date', 'precip_in']].rename(columns={'AirPtCd':'airptcd'})


In [13]:
hist_df

,airptcd,obs_date,precip_in
0,KBHB,2014-07-01,0.00
1,KBHB,2014-07-02,0.00
2,KBHB,2014-07-03,0.00
3,KBHB,2014-07-04,0.24
4,KBHB,2014-07-05,2.91
...,...,...,...
130452,PHNL,2017-08-28,0.00
130453,PHNL,2017-08-29,0.00
130454,PHNL,2017-08-30,0.00
130455,PHNL,2017-08-31,0.00


In [14]:
forecast_df['target_date'] = pd.to_datetime(forecast_df['target_date'])
forecast_df['forecast_issue_date'] = pd.to_datetime(forecast_df['forecast_issue_date'])

merged = forecast_df.merge(hist_df, left_on=['airptcd','target_date'], right_on=['airptcd','obs_date'], how='left')

# Binary rain outcome
merged['rain_outcome'] = (merged['precip_in'] > 0).astype(int)

# Convert forecast values to probability (0-1)
if merged['forecast_value'].max() <= 1.01:
    merged['forecast_prob'] = merged['forecast_value']
else:
    merged['forecast_prob'] = merged['forecast_value'] / 100.0

# Select relevant columns
merged_clean = merged[["target_date","city","state","airptcd",
                       "forecast_issue_date","forecast_var","forecast_prob",
                       "precip_in","rain_outcome"]]

In [16]:
merged_clean

,target_date,city,state,airptcd,forecast_issue_date,forecast_var,forecast_prob,precip_in,rain_outcome
0,2014-07-09,Eastport,Maine,KBHB,2014-07-09,ProbPrecip,0.0,0.00,0
1,2014-07-09,Eastport,Maine,KBHB,2014-07-09,ProbPrecip,0.0,0.00,0
2,2014-07-10,Eastport,Maine,KBHB,2014-07-09,ProbPrecip,0.0,0.03,1
3,2014-07-10,Eastport,Maine,KBHB,2014-07-09,ProbPrecip,0.0,0.03,1
4,2014-07-11,Eastport,Maine,KBHB,2014-07-09,ProbPrecip,0.0,0.00,0
...,...,...,...,...,...,...,...,...,...
1600659,2017-09-05,Honolulu,Hawaii,PHNL,2017-09-01,ProbPrecip,0.2,NaN,0
1600660,2017-09-05,Honolulu,Hawaii,PHNL,2017-09-01,ProbPrecip,0.2,NaN,0
1600661,2017-09-06,Honolulu,Hawaii,PHNL,2017-09-01,ProbPrecip,0.4,NaN,0
1600662,2017-09-06,Honolulu,Hawaii,PHNL,2017-09-01,ProbPrecip,0.5,NaN,0


In [17]:
# save merged df
merged_clean.to_csv(output_csv, index=False, date_format="%Y-%m-%d")